This notebook is intended to set the intial peak parameters.

## Imports

In [ ]:
import pathlib

from xrdfit.spectrum_fitting import PeakParams, FitSpectrum
import yaml

import analysis_functions as analysis


%load_ext autoreload
%autoreload 2

# Find data files
Rather than type in the full name of each file, it is easier to glob the files and then go through the list.

In [ ]:
data_folder = pathlib.Path("../example_data/caked")
files = list(data_folder.glob("*.dat"))

## Initial fit

Specify the name of the file to be analysed and give the angle of the first cake:

In [ ]:
file_path = files[0]
first_cake_angle = 90

Load the data:

In [ ]:
spectral_data = FitSpectrum(file_path, first_cake_angle)
spectral_data.plot_polar()

Choose the cakes to fit:

In [ ]:
cakes_to_fit = [27, 28, 29, 9, 10, 11]
spectral_data.highlight_cakes(cakes_to_fit)

Plot the spectra:

In [ ]:
spectral_data.plot(cakes_to_fit, x_range=(3.02, 3.5), show_points=True)

Set the peak parameters:

In [ ]:
peak_params = [PeakParams((2.75, 2.95), '110'),
               PeakParams((3.02, 3.32), ['200', '10-10', '10-11'], [(3.09, 3.12), (3.19, 3.22), (3.24, 3.26)]),
               PeakParams((4.13, 4.30), '210')]

Check the peak parameters are good

In [ ]:
spectral_data.plot_peak_params(peak_params, cakes_to_fit, show_points=True)
spectral_data.fit_peaks(peak_params, cakes_to_fit, merge_cakes=False)

spectral_data.plot_fit('110')
spectral_data.plot_fit('200')
spectral_data.plot_fit('210')

## Check instrument data

The `plot_instrument_data` function in `analysis_functions` can be used to load and plot the instrument data (ETMT analogue output) recorded with each of the diffraction pattern images.

Define a range covering the deformation (in steps of 100 ending with 1) and plot the instrument data:

In [ ]:
# define the file path to the instrument data 
file_path_instrument_data = "../example_data/image_load_files_for_DAWN/adc_065_TI64_NDload_900C_15mms.dat"

# set the range covering deformation
start_deform = 3101
end_deform = 3401

# the calculated max_frame will be used as the last step in a refinement
max_frame = analysis.plot_instrument_data(file_path_instrument_data, start_deform, end_deform)

# Save parameters to be used in the analysis

In [ ]:
params = {"data_file": str(file_path),
          "first_cake_angle": first_cake_angle,
          "cakes_to_fit": cakes_to_fit, 
          "peak_params": peak_params,
          "frame_time": 0.1,
          "merge_cakes": False,
          "start_deform": start_deform,
          "end_deform": end_deform,
          "max_frame": max_frame
         }

data_folder = f"../analysis/{file_path.stem}_params.yaml"
with open(data_folder, "w") as output_file:
    yaml.dump(params, output_file)